In [1]:
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
from urllib.request import Request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import os
import re

In [2]:
myurl =  'https://cedh-decklist-database.com'
cards = [] # initializes list of all cards

In [68]:

client = uReq(myurl) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("div",{"class": "ddb-section"})
htmls = []
for c in containers:
    if c.get_text().strip() == 'COMPETITIVE':
        x = c.parent.parent.find("ul", {"class": "ddb-decklists"})
        lis = x.findAll('li')
        for li in lis:
            htmls.append(li.a["href"])
len(htmls)

211

In [70]:
#cleaning moxfield primer lists
def second_group(m):
    return m.group(1)
htmls = list(map(lambda x: re.sub(r'(.*)/$|(.*)/primer$',second_group,x ), htmls))
htmls[128:131]

NameError: name 'htmls' is not defined

In [16]:
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.headless = True
for decklist in htmls[:4]:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        driver = webdriver.Chrome(chromedriver_path, options= options)
        driver.get(decklist)
        time.sleep(5) #if you want to wait 3 seconds for the page to load
        page_source = driver.page_source
        driver.close()
        page_soup = soup(page_source, 'html.parser')
        containers = page_soup.findAll("tr",{"class": "table-deck-row"})
        title = page_soup.find("span", {"class": "deckheader-name"}).get_text()
        deck_info = {'Link': decklist, 'Title':title }
        deck_cards = []
        for c in containers:
            c_name = {'Card Name': c.a.get_text()}
            deck_cards.append(c_name)
        deck_info['Cards'] = deck_cards
        cards.append(deck_info)

In [17]:
print(cards[0])

{'Link': 'https://www.moxfield.com/decks/1vqfLIEtWkOnZo-ywKQtKA', 'Title': 'Tymna/Sidar Raza', 'Cards': [{'Card Name': 'Sidar Kondo of Jamuraa'}, {'Card Name': 'Tymna the Weaver'}, {'Card Name': 'Arbor Elf'}, {'Card Name': "Avacyn's Pilgrim"}, {'Card Name': 'Birds of Paradise'}, {'Card Name': 'Cabal Therapist'}, {'Card Name': 'Collector Ouphe'}, {'Card Name': 'Corpse Knight'}, {'Card Name': 'Dark Confidant'}, {'Card Name': 'Dauthi Voidwalker'}, {'Card Name': 'Deathrite Shaman'}, {'Card Name': 'Drannith Magistrate'}, {'Card Name': 'Elesh Norn, Grand Cenobite'}, {'Card Name': 'Elves of Deep Shadow'}, {'Card Name': 'Elvish Mystic'}, {'Card Name': 'Esper Sentinel'}, {'Card Name': 'Fyndhorn Elves'}, {'Card Name': 'Grim Flayer'}, {'Card Name': 'Hope of Ghirapur'}, {'Card Name': 'Leonin Relic-Warder'}, {'Card Name': 'Llanowar Elves'}, {'Card Name': 'Loyal Retainers'}, {'Card Name': 'Opposition Agent'}, {'Card Name': 'Priest of Titania'}, {'Card Name': 'Razaketh, the Foulblooded'}, {'Card Name

In [18]:
df = pd.json_normalize(cards,
    record_path='Cards', meta=['Link', 'Title'])

In [20]:
df['Title'].value_counts()

Tymna/Sidar Raza                 111
[Primer] Rog Turbo Ad Nauseam    111
Frozen Scepter Parun 2.0         105
Niv-Mizzet, Parun                100
Name: Title, dtype: int64

In [84]:
df_1 = df.groupby('Card Name')['Title'].apply(list).reset_index()
df_2 = df.groupby('Card Name')['Link'].apply(list).reset_index()
df_12 = pd.merge(df_1, df_2, on='Card Name')
df_12.head()

,Card Name,Title,Link
0,Academy Ruins,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
1,Adarkar Wastes,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
2,Ancient Tomb,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
3,Arcane Signet,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
4,Archon of Emeria,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...


In [77]:
card_vc = df['Card Name'].value_counts()
card_vc = card_vc.reset_index()
card_vc.rename( columns={'index' :'Card Name', 'Card Name':'Ocurrences'}, inplace=True )#columns names
card_vc

,Card Name,Ocurrences
0,Chrome Mox,5
1,Lotus Petal,5
2,Mana Crypt,5
3,Mox Diamond,5
4,Jeweled Lotus,3
...,...,...
151,Sensei's Divining Top,1
152,Prismatic Vista,1
153,Timetwister,1
154,Waterlogged Grove,1


In [95]:
card_vc.to_json('results/competitiveCards.json', orient='records')
card_vc.to_csv('results/competitive_cards.csv')

In [85]:
full_cards = pd.merge(df_12,card_vc, on = 'Card Name')
full_cards.head()

,Card Name,Title,Link,Ocurrences
0,Academy Ruins,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1
1,Adarkar Wastes,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1
2,Ancient Tomb,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,2
3,Arcane Signet,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,2
4,Archon of Emeria,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1


In [86]:
full_cards.to_json('results/competitiveCards_full.json', orient='records')

In [96]:
my_collection = pd.read_csv('ArchidektCollection-Sat Jul 10 2021.csv')
my_collection.head()

,Quantity,Name
0,1,"Tasigur, the Golden Fang"
1,1,Witherbloom Apprentice
2,1,Emiel the Blessed
3,1,"Valentin, Dean of the Vein // Lisette, Dean of..."
4,1,Goblin Matron


In [97]:
sd = pd.merge(card_vc,my_collection, how='left', left_on='Card Name', right_on='Name', indicator=True)
sd[sd['_merge']=='left_only'].head(40)

,Card Name,Occurences,Quantity,Name,_merge
34,Imperial Seal,193,NaN,NaN,left_only
52,Timetwister,144,NaN,NaN,left_only
61,Wheel of Fortune,137,NaN,NaN,left_only
82,Volcanic Island,110,NaN,NaN,left_only
90,Badlands,106,NaN,NaN,left_only
93,Necropotence,102,NaN,NaN,left_only
96,Gaea's Cradle,99,NaN,NaN,left_only
121,Cavern of Souls,79,NaN,NaN,left_only
130,Savannah,75,NaN,NaN,left_only
135,Taiga,72,NaN,NaN,left_only


In [66]:
htmls = ['https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ','https://www.moxfield.com/decks/_4HeABv9DEeykmnZy1JICg', 'https://www.moxfield.com/decks/Re-_xtEbCkyNdH69vyGR8w','https://www.moxfield.com/decks/TnwluHJbcU2-Ioz8Op4txw']
# htmls = ['https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ']
cards = []
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.headless = True
driver = webdriver.Chrome(chromedriver_path, options= options)
for decklist in htmls:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        driver.get(decklist)
        time.sleep(3) #if you want to wait 3 seconds for the page to load
        mode = driver.find_element_by_id('viewMode').get_attribute('value')
        page_source = driver.page_source
        page_soup = soup(page_source, 'html.parser')
        title = page_soup.find("span", {"class": "deckheader-name"}).get_text()
        deck_info = {'Link': decklist, 'Title':title }
        if mode == 'table': 
            containers = page_soup.findAll("tr",{"class": "table-deck-row"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.a.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'visual':
            containers = page_soup.findAll("div",{"class": "decklist-card-phantomsearch"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'stacks':
            containers = page_soup.findAll("div",{"class": "img-card-stack"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.img['alt']}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)

df = pd.json_normalize(cards,
    record_path='Cards', meta=['Link', 'Title'])
df['Title'].value_counts()

Blood Farm            116
BAD Korvold (cEDH)    107
Consultation Zur      100
Marwyn, nerfed         73
Name: Title, dtype: int64

In [74]:
df = pd.read_json('results/competitiveCards_full.json')
df.nunique

<bound method DataFrame.nunique of                     Card Name  \
0              Abandon Reason   
1                    Abeyance   
2                      Abrade   
3                Abrupt Decay   
4                   Abundance   
...                       ...   
1549     Zirda, the Dawnwaker   
1550  Zndrsplt, Eye of Wisdom   
1551       Zulaport Cutthroat   
1552        Zur the Enchanter   
1553           Zur's Weirding   

                                                  Title  \
0                             [Filter Anje cEDH Primer]   
1                        [Heliod II: Sunshine Boogaloo]   
2     [Frozen Scepter Parun 2.0, Rielle Adaptative W...   
3     [Tymna/Sidar Raza, The King's Chain, Medium Mi...   
4                             [Simic Ad Naus in the CZ]   
...                                                 ...   
1549     [Dawnwaker Thrasios, Advantage Blue Pod (ABP)]   
1550           [🎲 50% of the Time, I Lose Every Time 🎲]   
1551  [Turbo Poison Hulk, Demoncratic

In [72]:
df = pd.read_json('results/competitiveCards.json')
df.head()

,Card Name,Ocurrences
0,Mana Crypt,406
1,Chrome Mox,404
2,Mox Diamond,400
3,Lotus Petal,316
4,Jeweled Lotus,247


In [53]:
scry = pd.read_json('results/scryfall_oracle.json')

cedh = pd.read_json('results/competitiveCards_full.json')

In [74]:
scry2 = scry[~scry['type_line'].str.contains(r'\bToken\b')]
scry3 = scry[scry['multiverse_ids'].str.len() != 0].copy()

In [75]:
scry3['name'].value_counts().sort_values(ascending=False).head(20)

Garbage Elemental             6
Sly Spy                       6
Very Cryptic Command          6
Knight of the Kitchen Sink    6
Ineffable Blessing            6
Everythingamajig              6
B.F.M. (Big Furry Monster)    2
Heart-Piercer Bow             1
Vaultbreaker                  1
Ogre Battledriver             1
Wall of Vapor                 1
Foul-Tongue Invocation        1
Returned Centaur              1
Granitic Titan                1
Maulfist Squad                1
Early Frost                   1
Curse of Fool's Wisdom        1
Oracle of Nectars             1
Pardic Lancer                 1
Sequence Engine               1
Name: name, dtype: int64

In [68]:
scry_cols = ['name', 'color_identity', 'type_line', 'reserved' , 'prices', 'mana_cost' ]
scry3[scry33'type_line'].str.contains('//))]['type_line'].head(50)

246     Creature — Werewolf Horror // Creature — Eldra...
373                                    Instant // Sorcery
409                                    Sorcery // Sorcery
494                    Land // Legendary Creature — Demon
497              Creature — Faerie // Sorcery — Adventure
693                                    Sorcery // Sorcery
775                                    Instant // Instant
807     Creature — Human Cleric // Creature — Eldrazi ...
840      Creature — Treefolk Druid // Sorcery — Adventure
962     Legendary Creature — God // Legendary Creature...
1022                                   Instant // Sorcery
1025        Legendary Creature — Human Warlock // Sorcery
1041                                     Artifact // Land
1093                                      Instant // Land
1225    Legendary Creature — Human Werewolf // Legenda...
1540      Creature — Zombie Knight // Instant — Adventure
1572      Creature — Zombie Knight // Instant — Adventure
1646       Cre

In [64]:
scry[scry['name'].str.contains('Jorn')][scry_cols].head(50)

,name,color_identity,type_line,reserved,prices,mana_cost
17489,"Jorn, God of Winter // Kaldring, the Rimestaff","[B, G, U]",Legendary Snow Creature — God // Legendary Sno...,False,"{'usd': '0.29', 'usd_foil': '0.45', 'eur': '0....",NaN
23580,"Jorn, God of Winter // Jorn, God of Winter",[],Card // Card,False,"{'usd': None, 'usd_foil': None, 'eur': None, '...",NaN


In [61]:
scry[scry['type_line']=='Card // Card']

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,promo_types,life_modifier,hand_modifier,loyalty,color_indicator,content_warning,printed_name,flavor_name,printed_type_line,printed_text
46,card,364535a0-fa83-4e27-8cce-b38481b5eff1,007e1815-b2ae-4861-b8c4-b0da8896d718,[],NaN,NaN,NaN,NaN,Professor of Zoomancy // Professor of Zoomancy,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,card,5d2aa3d3-657e-4088-9e00-db4554f3c270,00fc8d67-3df7-419a-9186-f05eb5cccb29,[],NaN,NaN,NaN,NaN,Purple Worm // Purple Worm,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,card,57c51bf1-9b6d-46cf-ac13-8e851c0efacf,0103cd34-9ce5-4569-9e6f-c583b58f555c,[],NaN,NaN,NaN,NaN,Prismari Campus // Prismari Campus,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,card,7677f694-e611-416d-8e9e-86d3b33b4c76,020e5b34-c546-4abe-a412-36aa1713b241,[],NaN,NaN,NaN,NaN,Wish // Wish,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,card,9e282d87-3890-468d-b4e3-c70504071d5f,024dab6c-3a77-4452-b1db-975f479dd818,[],NaN,NaN,NaN,NaN,Inga Rune-Eyes // Inga Rune-Eyes,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23580,card,9ba6e2ce-ea68-47e8-80f8-e932e0cb3a80,fdc53f90-90a3-4a29-8a47-2595809da8d3,[],NaN,NaN,NaN,NaN,"Jorn, God of Winter // Jorn, God of Winter",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23647,card,1a56de52-356b-4a22-84bd-f21cf853246e,fe8630df-983c-4049-b34b-13f263a71776,[],NaN,NaN,NaN,NaN,You Meet in a Tavern // You Meet in a Tavern,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23674,card,2ae53d84-208e-4db8-bf07-4026261601f7,febec943-9531-4ef9-b843-651bb540535a,[],NaN,NaN,NaN,NaN,Mind Flayer // Mind Flayer,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23710,card,6e42ae5d-aa09-4087-98ec-4da31a95b2a5,ff2580fa-5f8d-4c06-9930-84c66f4a09f0,[],NaN,NaN,NaN,NaN,Memory Lapse // Memory Lapse,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#cleaning type text lists
def second_group(m):
    return m.group(1)
scry3['type'] = scry3['type_line'].apply(lambda x: re.sub(r'.*(Creature|Land|Artifact|Instant|Sorcery|Planeswalker|Enchantment).*—*\/?',second_group,x))
scry3['type'].unique()

array(['Artifact', 'Enchantment', 'Sorcery', 'Creature', 'Instant',
       'Land', 'Scheme', 'Vanguard', 'Ongoing Scheme', 'Planeswalker',
       'Plane — Pyrulea', 'Conspiracy', 'Plane — Zendikar',
       'Plane — Ravnica', 'Plane — Dominaria', 'Plane — Ir',
       'Plane — Phyrexia', 'Eaturecray — Igpay', 'Plane — New Phyrexia',
       'Plane — Alara', 'Plane — Lorwyn', 'Plane — Innistrad',
       'Plane — Shandalar', 'Plane — Mongseng', 'Plane — Luvion',
       'Plane — Kyneth', 'Phenomenon', 'Summon Specter',
       'Plane — Wildfire', 'Plane — Fabacin', 'Plane — Mirrodin',
       'instant', 'Plane — Ulgrotha', 'Plane — Kaldheim',
       'Plane — Kamigawa', 'Plane — Xerex', 'Plane — Kolbahan',
       'Plane — Moag', 'Plane — Arkhos', 'Plane — Iquatana',
       'Summon — Dinosaur', 'Plane — Equilor', 'Plane — Azgol',
       'Plane — Muraganda', 'Plane — Mercadia', 'Plane — Vryn',
       'Plane — Shadowmoor', 'Plane — Karsus', 'Plane — Kinshala',
       'Plane — Belenon', 'Plane — Se

In [22]:
scry.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang',
       'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line',
       'oracle_text', 'colors', 'color_identity', 'keywords', 'legalities',
       'games', 'reserved', 'foil', 'nonfoil', 'oversized', 'promo', 'reprint',
       'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       'prints_search_uri', 'collector_number', 'digital', 'rarity',
       'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       'booster', 'story_spotlight', 'edhrec_rank', 'prices', 'related_uris',
       'preview', 'power', 'toughness', 'arena_id', 'watermark',
       'produced_mana', 'all_parts', 'card_faces', 'frame_effects',
       'promo_ty

In [56]:
scry['legalities']

0        {'standard': 'not_legal', 'future': 'not_legal...
1        {'standard': 'not_legal', 'future': 'not_legal...
2        {'standard': 'not_legal', 'future': 'not_legal...
3        {'standard': 'not_legal', 'future': 'not_legal...
4        {'standard': 'not_legal', 'future': 'not_legal...
                               ...                        
23794    {'standard': 'not_legal', 'future': 'not_legal...
23795    {'standard': 'not_legal', 'future': 'not_legal...
23796    {'standard': 'legal', 'future': 'legal', 'hist...
23797    {'standard': 'not_legal', 'future': 'not_legal...
23798    {'standard': 'not_legal', 'future': 'not_legal...
Name: legalities, Length: 23799, dtype: object

In [29]:
cedh_scry = pd.merge(cedh,scry[scry_cols], how='left', left_on='Card Name', right_on='name', indicator=True)

In [30]:
cedh_scry[cedh_scry['_merge']=='left_only']

,Card Name,Title,Link,Ocurrences,name,color_identity,type_line,reserved,prices,_merge
17,Agadeem's Awakening,"[The King's Chain, Yuriko Pivot, cEDH Inalla -...",[https://www.moxfield.com/decks/uxtkikNzW0mTlu...,7,NaN,NaN,NaN,NaN,NaN,left_only
92,Barkchannel Pathway,"[[cEDH] Momir Vig's Hackball, Rashmi Draw Go]",[https://www.moxfield.com/decks/tIL7CthM5kuTXk...,2,NaN,NaN,NaN,NaN,NaN,left_only
110,"Birgi, God of Storytelling","[Frozen Scepter Parun 2.0, Rakdos Adept, celeb...",[https://www.moxfield.com/decks/belqftDqDEWVEZ...,30,NaN,NaN,NaN,NaN,NaN,left_only
121,Blightstep Pathway,[DocksideDoomRound],[https://www.moxfield.com/decks/jU1S6wncyUmNA3...,1,NaN,NaN,NaN,NaN,NaN,left_only
157,Branchloft Pathway,"[Tymna/Sidar Raza, Selvala Twiddlestorm cEDH P...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,2,NaN,NaN,NaN,NaN,NaN,left_only
226,Clearwater Pathway,"[Kels Consultation Scepter cEDH, Don't Talk Ab...",[https://www.moxfield.com/decks/5_b4DltiAkaMGD...,5,NaN,NaN,NaN,NaN,NaN,left_only
262,Cragcrown Pathway,[Snoop Tharry Tharr [Primer]: Gruul Was the Ha...,[https://www.moxfield.com/decks/wHkTll0w6EuJPk...,1,NaN,NaN,NaN,NaN,NaN,left_only
296,Darkbore Pathway,"[Tymna/Sidar Raza, Turbo Poison Hulk, Honorles...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,4,NaN,NaN,NaN,NaN,NaN,left_only
387,Emeria's Call,"[Heliod, Stax-Crowned cEDH, Heliod II: Sunshin...",[https://www.moxfield.com/decks/urs4pKTeCUyZ-F...,8,NaN,NaN,NaN,NaN,NaN,left_only
448,Flamescroll Celebrant,"[Clampstorm, 💧 Blue Farm💧 , [cEDH] Return of t...",[https://www.moxfield.com/decks/QP1PcYDAnkCv39...,3,NaN,NaN,NaN,NaN,NaN,left_only


In [31]:
cedh_scry.head()

,Card Name,Title,Link,Ocurrences,name,color_identity,type_line,reserved,prices,_merge
0,Abandon Reason,[Filter Anje cEDH Primer],[https://www.moxfield.com/decks/PFGBjM5BnESfH9...,1,Abandon Reason,[R],Instant,False,"{'usd': '0.10', 'usd_foil': '0.26', 'eur': '0....",both
1,Abeyance,[Heliod II: Sunshine Boogaloo],[https://www.moxfield.com/decks/rohOeIs-dEmA1L...,1,Abeyance,[W],Instant,True,"{'usd': '31.10', 'usd_foil': None, 'eur': '3.9...",both
2,Abrade,"[Frozen Scepter Parun 2.0, Rielle Adaptative W...",[https://www.moxfield.com/decks/belqftDqDEWVEZ...,34,Abrade,[R],Instant,False,"{'usd': '0.45', 'usd_foil': None, 'eur': '0.24...",both
3,Abrupt Decay,"[Tymna/Sidar Raza, The King's Chain, Medium Mi...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,65,Abrupt Decay,"[B, G]",Instant,False,"{'usd': '6.13', 'usd_foil': '14.48', 'eur': '5...",both
4,Abundance,[Simic Ad Naus in the CZ],[https://www.moxfield.com/decks/Vva2blXPDEWTPC...,1,Abundance,[G],Enchantment,False,"{'usd': '0.23', 'usd_foil': None, 'eur': '0.20...",both


In [32]:
cedh_scry2 = pd.read_json('results/cedh_scry.json')

In [33]:
cedh_scry2[cedh_scry2['_merge']=='left_only']

,Card Name,Title,Link,Ocurrences,name,color_identity,type_line,reserved,prices,_merge
17,Agadeem's Awakening,"[The King's Chain, Yuriko Pivot, cEDH Inalla -...",[https://www.moxfield.com/decks/uxtkikNzW0mTlu...,7,"Agadeem's Awakening // Agadeem, the Undercrypt",[B],Sorcery // Land,False,"{'usd': '14.65', 'usd_foil': '15.08', 'eur': '...",left_only
92,Barkchannel Pathway,"[[cEDH] Momir Vig's Hackball, Rashmi Draw Go]",[https://www.moxfield.com/decks/tIL7CthM5kuTXk...,2,Barkchannel Pathway // Tidechannel Pathway,"[G, U]",Land // Land,False,"{'usd': '5.48', 'usd_foil': '5.39', 'eur': '3....",left_only
110,"Birgi, God of Storytelling","[Frozen Scepter Parun 2.0, Rakdos Adept, celeb...",[https://www.moxfield.com/decks/belqftDqDEWVEZ...,30,"Birgi, God of Storytelling // Harnfel, Horn of...",[R],Legendary Creature — God // Legendary Artifact,False,"{'usd': '5.26', 'usd_foil': '5.87', 'eur': '2....",left_only
121,Blightstep Pathway,[DocksideDoomRound],[https://www.moxfield.com/decks/jU1S6wncyUmNA3...,1,Blightstep Pathway // Searstep Pathway,"[B, R]",Land // Land,False,"{'usd': '5.13', 'usd_foil': '5.19', 'eur': '4....",left_only
157,Branchloft Pathway,"[Tymna/Sidar Raza, Selvala Twiddlestorm cEDH P...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,2,Branchloft Pathway // Boulderloft Pathway,"[G, W]",Land // Land,False,"{'usd': '2.31', 'usd_foil': '2.90', 'eur': '3....",left_only
226,Clearwater Pathway,"[Kels Consultation Scepter cEDH, Don't Talk Ab...",[https://www.moxfield.com/decks/5_b4DltiAkaMGD...,5,Clearwater Pathway // Murkwater Pathway,"[B, U]",Land // Land,False,"{'usd': '5.13', 'usd_foil': '4.85', 'eur': '4....",left_only
262,Cragcrown Pathway,[Snoop Tharry Tharr [Primer]: Gruul Was the Ha...,[https://www.moxfield.com/decks/wHkTll0w6EuJPk...,1,Cragcrown Pathway // Timbercrown Pathway,"[G, R]",Land // Land,False,"{'usd': '2.83', 'usd_foil': '2.64', 'eur': '2....",left_only
296,Darkbore Pathway,"[Tymna/Sidar Raza, Turbo Poison Hulk, Honorles...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,4,Darkbore Pathway // Slitherbore Pathway,"[B, G]",Land // Land,False,"{'usd': '8.37', 'usd_foil': '8.92', 'eur': '6....",left_only
387,Emeria's Call,"[Heliod, Stax-Crowned cEDH, Heliod II: Sunshin...",[https://www.moxfield.com/decks/urs4pKTeCUyZ-F...,8,"Emeria's Call // Emeria, Shattered Skyclave",[W],Sorcery // Land,False,"{'usd': '2.03', 'usd_foil': '2.31', 'eur': '2....",left_only
448,Flamescroll Celebrant,"[Clampstorm, 💧 Blue Farm💧 , [cEDH] Return of t...",[https://www.moxfield.com/decks/QP1PcYDAnkCv39...,3,Flamescroll Celebrant // Revel in Silence,"[R, W]",Creature — Human Shaman // Instant,False,"{'usd': '0.22', 'usd_foil': '0.26', 'eur': '0....",left_only
